## Fraud Detection

## Fraud Detection

#### Importing necessary libraries

In [6]:
import os 
import sys
import pandas as pd
import sidetable as stb
import seaborn as sns
import matplotlib.pyplot as plt
from mlflow import MlflowClient
from pprint import pprint

In [2]:
sys.path.append(os.path.abspath('../scripts'))

from Utils import DataUtils
from  Plotting import Plots

utils = DataUtils()
plots = Plots()

In [4]:
fraud_data = utils.load_data('Clean_FraudData.csv')

2024-10-22 02:59 - DEBUG :: Loading data from file...
2024-10-22 02:59 - INFO :: Loading Clean_FraudData.csv took 5.26 seconds




In [8]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

#### Logistic regression Model

In [10]:
experiment_description = (
    "This is the Fraud forcasting project. "
    "This experiment contains the produce models for predicting frauds using logistic regression."
)


experiment_tags = {
    "project_name": "Fraud-Data-forecasting",
    "mlflow.note.content": experiment_description,
}

produce_apples_experiment = client.create_experiment(
    name="Fraud_Models", tags=experiment_tags
)